In [1]:
import os
import json
import io
from data_generator import DataGenerator
from trade_env import TraderEnv

In [2]:
dg = DataGenerator()

In [3]:
trade = TraderEnv(dg, stage_history_length=20)

In [4]:
import threading
import numpy as np
import tensorflow as tf
import pylab
import time
import gym
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K


# global variables for threading
episode = 0
scores = []

EPISODES = 10000

# This is A3C(Asynchronous Advantage Actor Critic) agent(global) for the Cartpole
# In this example, we use A3C algorithm
class A3CAgent:
    def __init__(self, state_size, action_size, env_name):
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # get gym environment name
        self.env_name = env_name

        # these are hyper parameters for the A3C
        self.actor_lr = 0.00002
        self.critic_lr = 0.00002
        self.discount_factor = .6

        self.threads = 32

        # create model for actor and critic network
        self.actor, self.critic = self.build_model()

        # method for training actor and critic network
        self.optimizer = [self.actor_optimizer(), self.critic_optimizer()]

        self.sess = tf.InteractiveSession()
        K.set_session(self.sess)
        self.sess.run(tf.global_variables_initializer())

    # approximate policy and value using Neural Network
    # actor -> state is input and probability of each action is output of network
    # critic -> state is input and value of state is output of network
    # actor and critic network share first hidden layer
    def build_model(self):
        state = Input(batch_shape=(None,  self.state_size))
        shared = Dense(self.state_size*2, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_uniform')(state)
        
        actor_hidden = Dense(self.state_size, activation='relu', kernel_initializer='glorot_uniform')(shared)
        actor_hidden = Dense(60, activation='relu', kernel_initializer='glorot_uniform')(actor_hidden)
        actor_hidden = Dense(30, activation='relu', kernel_initializer='glorot_uniform')(actor_hidden)
        actor_hidden = Dense(20, activation='relu', kernel_initializer='glorot_uniform')(actor_hidden)
        action_prob = Dense(self.action_size, activation='softmax', kernel_initializer='glorot_uniform')(actor_hidden)

        value_hidden = Dense(self.state_size, activation='relu', kernel_initializer='he_uniform')(shared)
        value_hidden = Dense(80, activation='relu', kernel_initializer='he_uniform')(value_hidden)
        value_hidden = Dense(50, activation='relu', kernel_initializer='he_uniform')(value_hidden)
        value_hidden = Dense(25, activation='relu', kernel_initializer='he_uniform')(value_hidden)
        state_value = Dense(1, activation='linear', kernel_initializer='he_uniform')(value_hidden)

        actor = Model(inputs=state, outputs=action_prob)
        critic = Model(inputs=state, outputs=state_value)

        actor._make_predict_function()
        critic._make_predict_function()

        #actor.summary()
        #critic.summary()

        return actor, critic

    # make loss function for Policy Gradient
    # [log(action probability) * advantages] will be input for the back prop
    # we add entropy of action probability to loss
    def actor_optimizer(self):
        action = K.placeholder(shape=(None, self.action_size))
        advantages = K.placeholder(shape=(None, ))

        policy = self.actor.output

        good_prob = K.sum(action * policy, axis=1)
        eligibility = K.log(good_prob + 1e-10) * K.stop_gradient(advantages)
        loss = -K.sum(eligibility)

        entropy = K.sum(policy * K.log(policy + 1e-10), axis=1)

        actor_loss = loss + 0.01*entropy

        optimizer = Adam(lr=self.actor_lr)
        updates = optimizer.get_updates(self.actor.trainable_weights, [], actor_loss)
        train = K.function([self.actor.input, action, advantages], [], updates=updates)
        return train

    # make loss function for Value approximation
    def critic_optimizer(self):
        discounted_reward = K.placeholder(shape=(None, ))

        value = self.critic.output

        loss = K.mean(K.square(discounted_reward - value))

        optimizer = Adam(lr=self.critic_lr)
        updates = optimizer.get_updates(self.critic.trainable_weights, [], loss)
        train = K.function([self.critic.input, discounted_reward], [], updates=updates)
        return train

    # make agents(local) and start training
    def train(self):
        # self.load_model('./save_model/cartpole_a3c.h5')
        agents = [Agent(i, self.actor, self.critic, self.optimizer, self.env_name, self.discount_factor,
                        self.action_size, self.state_size) for i in range(self.threads)]

        for agent in agents:
            agent.start()

        for agent in agents:
            agent.join()

        plot = scores[:]
        pylab.plot(range(len(plot)), plot, 'b')
        pylab.savefig("./save_graph/model.png")

        self.save_model('./save_model/model')

    def save_model(self, name):
        self.actor.save_weights(name + "_actor.h5")
        self.critic.save_weights(name + "_critic.h5")

    def load_model(self, name):
        self.actor.load_weights(name + "_actor.h5")
        self.critic.load_weights(name + "_critic.h5")

# This is Agent(local) class for threading
class Agent(threading.Thread):
    def __init__(self, index, actor, critic, optimizer, env_name, discount_factor, action_size, state_size):
        threading.Thread.__init__(self)

        self.states = []
        self.rewards = []
        self.actions = []

        self.index = index
        self.actor = actor
        self.critic = critic
        self.optimizer = optimizer
        self.env_name = env_name
        self.discount_factor = discount_factor
        self.action_size = action_size
        self.state_size = state_size

    # Thread interactive with environment
    def run(self):
        global episode
        env = TraderEnv(DataGenerator(random=False, first_index=0))
        while episode < EPISODES:
            state = env.reset()
            score = 0
            
            while True:
                action = self.get_action(state)
                next_state, reward, done, _ = env.step(action)
                score += reward

                self.memory(state, action, reward)

                state = next_state

                if done:
                    episode += 1
                    print("episode: ", episode, "/ score : ", score)
                    scores.append(score)
                    self.train_episode(score < 0)
                    break
        print("Train finished: ", episode, " Thread: ", self.index)

    # In Policy Gradient, Q function is not available.
    # Instead agent uses sample returns for evaluating policy
    def discount_rewards(self, rewards, done=True):
        discounted_rewards = np.zeros_like(rewards)
        running_add = 0
        if not done:
            running_add = self.critic.predict(np.reshape(self.states[-1], (1, self.state_size)))[0]
        for t in reversed(range(0, len(rewards))):
            running_add = running_add * self.discount_factor + rewards[t]
            discounted_rewards[t] = running_add
        return discounted_rewards

    # save <s, a ,r> of each step
    # this is used for calculating discounted rewards
    def memory(self, state, action, reward):
        self.states.append(state)
        act = np.zeros(self.action_size)
        act[action] = 1
        self.actions.append(act)
        self.rewards.append(reward)

    # update policy network and value network every episode
    def train_episode(self, done):
        discounted_rewards = self.discount_rewards(self.rewards, done)

        values = self.critic.predict(np.array(self.states))
        values = np.reshape(values, len(values))

        advantages = discounted_rewards - values

        self.optimizer[0]([self.states, self.actions, advantages])
        self.optimizer[1]([self.states, discounted_rewards])
        self.states, self.actions, self.rewards = [], [], []

    def get_action(self, state):
        policy = self.actor.predict(np.reshape(state, [1, self.state_size]))[0]
        return np.random.choice(self.action_size, 1, p=policy)[0]




/Users/samirantonio/anaconda/envs/python3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [5]:
env = TraderEnv(DataGenerator())

state_size = env.observation_space.shape[0]
action_size = env.action_space.n


In [6]:
global_agent = A3CAgent(state_size, action_size, "TraderEnv")
global_agent.train()

Exception in thread Thread-34:
Traceback (most recent call last):
  File "/Users/samirantonio/anaconda/envs/python3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1323, in _do_call
    return fn(*args)
  File "/Users/samirantonio/anaconda/envs/python3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1302, in _run_fn
    status, run_metadata)
  File "/Users/samirantonio/anaconda/envs/python3/lib/python3.6/site-packages/tensorflow/python/framework/errors_impl.py", line 473, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.NotFoundError: FetchOutputs node IsVariableInitialized_7/IsVariableInitialized_76:0: not found

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/samirantonio/anaconda/envs/python3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "<ipython-input-4-54fd6f5cc504>", line 166, in run
 

Exception in thread Thread-25:
Traceback (most recent call last):
  File "/Users/samirantonio/anaconda/envs/python3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1323, in _do_call
    return fn(*args)
  File "/Users/samirantonio/anaconda/envs/python3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1302, in _run_fn
    status, run_metadata)
  File "/Users/samirantonio/anaconda/envs/python3/lib/python3.6/site-packages/tensorflow/python/framework/errors_impl.py", line 473, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.NotFoundError: FetchOutputs node IsVariableInitialized_4/IsVariableInitialized_77/IsVariableInitialized/IsVariableInitialized_66:0: not found

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/samirantonio/anaconda/envs/python3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "<ipy

episode:  1episode: episode: episode: episode:      episode: / score : 3245 episode: episode:      7  episode: episode: 8.899000000000001/ score : / score : / score : / score :  68  episode: episode: episode: episode: 
    / score :   910    148.8990000000000015.6998.6976.897 / score : / score :   111312 



8.498  / score : / score :    / score : / score : / score : / score :  
6.6994.697     6.6958.497

5.8956.29899999999999958.2995.898999999999999





episode:  15 / score :  6.097999999999999
episode:  16 / score :  8.693
episode:  episode:  18 / score : 17episode:    6.895/ score : 19
  episode: episode:  5.695/ score :  21
 20  8.293999999999999/ score : / score : 
  8.6939999999999998.697

episode:  22 / score :  6.699999999999999
episode:  episode: 23 24  episode: / score : / score :    258.38.5 

/ score :  8.9
episode:  26episode:   / score : 27  8.5/ score : 
 6.300000000000001
episode:  28 / score : episode:   8.529
 episode:  / score : 30  6.699999999999999/ score : 
 8.1


episode:  256 / score :  -95.0
episode:  257 / score :  -95.0
episode:  258episode:  / score :   -95.0259
 episode: / score :  -95.0 
260 / score :  -95.0
episode:  261 / score :  -95.0episode: 
 262 / score :  -95.0
episode:  263 / score :  -95.0
episode:  264 / score :  -95.0
episode:  265 / score :  -95.0
episode:  266 / score :  -95.0episode: episode: 
  267 / score : 268  / score : -95.0
 -95.0
episode:  269 / score : episode:   -95.0270
 / score :  -95.0
episode:  episode:  272 271/ score :   -95.0/ score : 
 -95.0
episode:  273 / score :  -95.0
episode:  274 / score :  -95.0
episode:  275 / score : episode:   -95.0
276 / score :  -95.0episode: 
 277 / score :  8.299999999999999
episode:  278 / score :  -95.0
episode:  279 / score :  -95.0
episode:  280 / score :  -95.0
episode:  281 / score :  -95.0
episode:  282 / score :  -95.0
episode: episode:  284  283/ score :   / score : -95.0 
-95.0
episode:  285 / score :  -95.0
episode:  286 / score :  episode: -95.0
 287 / score :  -9

episode:  519 / score :  -95.0
episode:  520 / score :  -95.0
episode:  episode:  522521  / score : / score :   -95.0-95.0

episode:  523 / score :  -95.0
episode:  524 / score :  -95.0episode: 
 525 / score :  -95.0
episode:  526 / score :  episode: -95.0 527
 / score :  -95.0
episode:  528 / score :  -95.0
episode:  529 / score :  -95.0
episode:  530 / score :  -95.0
episode:  531 / score :  -95.0
episode:  532 / score :  9.1
episode:  533 / score :  episode: -95.0
 534 / score : episode:  -95.0
 535 / score :  episode: -95.0 
536 / score :  -95.0
episode:  537 / score :  -95.0
episode: episode:  538  / score : 539  -95.0/ score : 
 -95.0
episode:  540 / score :  -95.0episode: 
 541 / score :  -95.0
episode: episode:  543 542  episode:  / score : / score : 544   -95.0-95.0/ score : 

 -95.0
episode:  545 / score :  -95.0
episode:  546 / score :  -95.0
episode:  547 / score :  -95.0
episode:  548 / score :  -95.0
episode:  549 / score :  -95.0
episode: episode:   551 / score : 550  -9

...

episode: episode:  8840  / score : 8841 -95.0 episode: 
/ score :  8842 -95.0 
/ score :  -95.0
episode:  8843 / score :  -95.0
episode:  8844 episode: / score :   -95.08845
 / score :  -95.0episode: episode:   episode: 
8846 8847 8848 / score :  / score :  / score :  -95.0 -95.0
-95.0

episode: episode:  8849 / score :   8850-95.0 
/ score :  -95.0
episode:  8851 / score :  -95.0
episode:  episode: episode: 8852 episode:  8854  / score :  88538855 / score :   -95.0 / score : / score : 
-95.0  -95.0
-95.0

episode:  episode: 8856  8857 / score :  -95.0/ score : 
 -95.0
episode:  8858 / score :  -95.0
episode: episode:   88608859  / score : / score :   -95.08.7

episode:  8861episode:   8862 / score : / score :   -95.0-95.0

episode:  8863 episode: / score :   8864-95.0 
/ score :  -95.0
episode:  8865episode:   / score : 8866  -95.0/ score : episode: 
  8867-95.0 
episode: / score :   8868-95.0 
/ score :  -95.0
episode:  8869 / score :  episode: -95.0 8870episode:  
8871  / score : / 

episode:  episode:  90929093  / score : / score :   -95.07.499999999999998

episode:  9094 episode: / score :   -95.09095
 episode: episode: episode:  / score :   9098 90979096 -95.0  / score : 
/ score : / score :    -95.09.18.299999999999999


episode:  episode: 9099  / score : 9100episode:    7.299999999999999/ score : 9101
  / score : 9.1 
episode: -95.0episode:  episode: 
 9102 episode: 91039104    / score : 9105/ score : / score :     -95.0/ score : -95.0-95.0
 

-95.0
episode:  9106 / score :  -95.0
episode:  9107 / score :  8.7
episode:  9108 / score : episode:   episode: 9109-95.0  
9110/ score :   / score : -95.0 
8.299999999999999
episode: episode:   91129111  / score :  -95.0
/ score :  -95.0
episode:  9113 / score :  -95.0
episode:  9114 / score :  7.499999999999998
episode:  9115 / score :  -95.0
episode: episode:  9116 9117  / score : / score :   episode: episode: -95.0-95.0  

91199118episode:   / score :   9120episode: -95.0/ score :   
 / score : 9121 7.09999999999999

 9332 / score : episode:  7.299999999999999 
9333 / score : episode:   -95.09334
 / score :  8.299999999999999
episode: episode:   93369335  / score : / score :   episode: 7.4999999999999988.1 

9337 / score :  7.499999999999998
episode: episode:  9338  9339/ score :   / score : 7.099999999999998 
-95.0
episode: episode:  9340  / score : 9341  / score :  7.699999999999999episode: 7.499999999999998
 
9342 / score :  episode: 7.499999999999998 
9343 / score : episode:   8.99344 
/ score :  7.899999999999999
episode:  9345 episode:  / score :  9346episode: 7.899999999999999 
/ score : episode:    9.193479348
  episode: / score : / score :   episode:  93498.7 7.099999999999998 
9350
/ score :  / score :   8.78.5

episode:  episode: 9351  9352 / score : / score :   9.18.299999999999999

episode:  9353 / score :  8.1
episode:  9354 / score :  8.9
episode:  9355 episode: / score :   9356-95.0 
/ score :  7.499999999999998
episode:  9357 / score :  7.099999999999998
episode:  9358 / score :  8

episode:  9564episode: episode:    95669565/ score :    / score : / score : episode: 7.699999999999999   
8.59.19567 

/ score : episode:   7.8999999999999999568
 / score :  episode: 7.699999999999999 
9569 / score :  8.299999999999999
episode: episode:   9571 / score :  9570episode: 8.7  
/ score : 9572  / score : 7.699999999999999 
8.9
episode:  9573 / score :  -95.0
episode:  9574 / score :  9.1
episode:  9575 / score :  8.9
episode:  9576episode:  episode: / score :  8.5
  9578 / score :  9.1episode: 9577
  / score : 9579episode:    episode: 8.9/ score : 9580 episode: 
  9581 episode:  8.9/ score :  95829583
/ score :  episode: episode:     / score :  / score : 95858.18.1 9584episode:   

8.1  8.5/ score : 9586/ score : 

   7.8999999999999999.1/ score : 

 9.1
episode:  9587 / score : episode:   7.6999999999999999588
 / score :  8.7
episode:  episode:  95899590 episode: episode:  / score :   / score :  95919592 8.1  7.699999999999999
/ score : / score : 
  8.97.899999999999999epis

episode:  episode: 9806  9807/ score :   / score : episode: episode: 7.899999999999999   episode: 
8.798089809 9810
   / score : / score : / score :    8.79.18.1

episode: 
episode:  9811  9812/ score :   7.899999999999999
/ score :  8.299999999999999
episode:  9813 / score :  8.7
episode:  9814 / score :  8.1
episode:  9815 / score :  8.7
episode:  episode: episode:  98169818  9817 / score :  / score : episode:  / score :   episode: 8.299999999999999 8.19819 
8.9
 9820 
/ score : / score :   7.8999999999999998.9

episode:  9821 / score :  8.5
episode:  episode:  9822 9823/ score :   / score : 8.9
 8.299999999999999
episode:  9824 / score :  9.1episode: episode: 
 9825 episode: / score :    982798268.9  
/ score : / score :   8.58.1

episode:  9828 / score : episode:   8.79829
 episode:  / score : 9830  9.1/ score : 
 7.699999999999999
episode:  9831 / score :  8.7
episode:  9832 / score :  8.7episode: 
 9833 / score : episode:  9.1
episode: episode:  episode: episode:   9834  98369835

In [7]:
def get_action(state):
    policy =  global_agent.actor.predict(np.reshape(state, [1, state_size]))[0]
    return np.argmax(policy)

In [8]:
env = TraderEnv(DataGenerator())
state = env.reset()

In [9]:
get_action(state)

1

In [10]:
env = TraderEnv(DataGenerator(random=False, first_index=0))
state = env.reset()
score = 0
while True:
    action = get_action(state)
    print(action)
    next_state, reward, done, info = env.step(action)
    score += reward

    state = next_state
    if done:
        print ("score: ", score , "info", info)
        break

1
1
1
1
1
1
1
1
1
1
1
1
2
2
score:  7.699999999999999 info {'status': 'Order sold'}


In [11]:
env._data_generator.first_index

0